# Audio Transcription and Speaker Clustering

In [1]:
!pip install pyannote.audio

In [7]:
from pyannote.audio import Pipeline
import torch
import os

In [6]:
token = "YOUR_TOKEN"
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=token)

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

In [9]:
!wget https://github.com/iramamy/SpeechToText/raw/main/data.mp3

--2024-07-02 13:08:34--  https://github.com/iramamy/SpeechToText/raw/main/data.mp3
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/iramamy/SpeechToText/main/data.mp3 [following]
--2024-07-02 13:08:34--  https://raw.githubusercontent.com/iramamy/SpeechToText/main/data.mp3
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36887489 (35M) [application/octet-stream]
Saving to: ‘data.mp3’

data.mp3            100%[===================>]  35.18M   217MB/s    in 0.2s    

2024-07-02 13:08:36 (217 MB/s) - ‘data.mp3’ saved [36887489/36887489]



# Diarization

#### Time for inference ~ 5 minutes (T4 GPU)

In [10]:
HOME = os.getcwd()

pipeline.to(torch.device("cuda"))
diarization = pipeline(f"{HOME}/data.mp3")

# print the result
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

start=5.5s stop=8.1s speaker_SPEAKER_01
start=8.2s stop=18.2s speaker_SPEAKER_06
start=18.4s stop=24.5s speaker_SPEAKER_01
start=25.9s stop=29.8s speaker_SPEAKER_02
start=32.2s stop=32.5s speaker_SPEAKER_01
start=33.2s stop=35.0s speaker_SPEAKER_02
start=34.5s stop=36.0s speaker_SPEAKER_01
start=37.1s stop=39.2s speaker_SPEAKER_02
start=39.2s stop=40.7s speaker_SPEAKER_02
start=39.3s stop=39.6s speaker_SPEAKER_01
start=40.5s stop=40.8s speaker_SPEAKER_01
start=41.2s stop=42.2s speaker_SPEAKER_02
start=42.2s stop=42.5s speaker_SPEAKER_06
start=42.5s stop=42.8s speaker_SPEAKER_02
start=42.8s stop=43.3s speaker_SPEAKER_06
start=43.3s stop=52.5s speaker_SPEAKER_01
start=52.7s stop=53.6s speaker_SPEAKER_05
start=53.4s stop=53.5s speaker_SPEAKER_01
start=53.6s stop=62.4s speaker_SPEAKER_01
start=55.0s stop=59.7s speaker_SPEAKER_06
start=62.7s stop=80.2s speaker_SPEAKER_06
start=77.0s stop=77.8s speaker_SPEAKER_02
start=77.8s stop=77.9s speaker_SPEAKER_00
start=80.7s stop=82.4s speaker_SPEAKE

In [11]:
num_speakers = 6

language = 'English'
model_size = 'medium'

model_name = model_size
if language == 'English' and model_size != 'large':
    model_name += '.en'


In [19]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Install whisper

In [20]:
%%capture
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-7mxfpdt6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-7mxfpdt6
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802823 sha256=7e684eb92134205b2450cecda63a03e498ad5288df28667d8d2f9d11634f26a6
  Stored in directory: /tmp/pip-ephem-wheel-cache-h8mqgn8z/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [28]:
import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np
import torchaudio

## Load the model

In [22]:
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 1.42G/1.42G [00:18<00:00, 84.4MiB/s]


## Transcribe the Audio File

In [23]:
result = model.transcribe(f"{HOME}/data.mp3")
segments = result["segments"]

In [37]:
audio, sample_rate = torchaudio.load(f"{HOME}/data.mp3")
duration = audio.shape[1] / sample_rate  # duration in seconds

## Compute Segment Embeddings

In [31]:
audio = Audio()

def segment_embedding(segment):
    start = segment["start"]
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(f"{HOME}/data.mp3", clip)
    return embedding_model(waveform[None])

In [32]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
    embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [33]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
    segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [35]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

## Write the final output to a text file

In [36]:
f = open(f"{HOME}/transcript.txt", "w")

for (i, segment) in enumerate(segments):
    if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
        f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
    f.write(segment["text"][1:] + ' ')

f.close()